In [1]:
#Imports needed

import quandl
import pandas as pd
import numpy as np
import os
from pathlib import Path
import datetime

In [2]:
# Setting the quandl API key variable within the quandl class
quandl.ApiConfig.api_key = os.getenv("QUANDL_API_KEY")

#getting the Sentiment Data from quandl
sentiment_data = pd.DataFrame(quandl.get(["AAII/AAII_SENTIMENT.1", "AAII/AAII_SENTIMENT.2", "AAII/AAII_SENTIMENT.3"], start_date = "2018-01-01", end_date = "2019-12-31"))#, start_date = "1987-07-24", end_date = "2020-01-20")#['Date', 'Bullish', 'Neutral', 'Bearish']
sentiment_data.columns = ['Bullish', 'Neutral', 'Bearish']
sentiment_data.reset_index(inplace = True)
sentiment_data.head()

Date   Bullish   Neutral   Bearish
0 2018-01-04  0.597531  0.246914  0.155556
1 2018-01-11  0.486726  0.262537  0.250737
2 2018-01-18  0.541133  0.244973  0.213894
3 2018-01-25  0.454545  0.305195  0.240260
4 2018-02-01  0.447712  0.264706  0.287582

In [3]:
sentiment_data['Bullish'] = sentiment_data['Bullish'].astype(float)
sentiment_data['Neutral'] = sentiment_data['Neutral'].astype(float)
sentiment_data['Bearish'] = sentiment_data['Bearish'].astype(float)
sentiment_data.dtypes

Date       datetime64[ns]
Bullish           float64
Neutral           float64
Bearish           float64
dtype: object

In [4]:
#Reading S&P500 Data
csvpath = Path("SP500.csv")
sp500_data = pd.read_csv(csvpath)
sp500_data['Date'] = pd.to_datetime(sp500_data['Date'])
sp500_data['Close'] = sp500_data['Close'].astype(float)
sp500_data.set_index('Date', drop = True, inplace = True)
sp500_data.head()

Close
Date              
1987-07-24  309.27
1987-07-27  310.65
1987-07-28  312.33
1987-07-29  315.65
1987-07-30  318.05

In [8]:
#Calculating S&P500 Daily changes in % (1 Day)
sp500_data['%Change'] = sp500_data.pct_change(1)
sp500_data.head(10)
aux_df = sp500_data

In [9]:
#Calculating Triggers
max_Bu = sentiment_data['Bullish'].max()
min_Bu = sentiment_data['Bullish'].min()
mean_Bu = sentiment_data['Bullish'].mean()
mid_Bu = (max_Bu + mean_Bu)/2

max_Be = sentiment_data['Bearish'].max()
min_Be = sentiment_data['Bearish'].min()
mean_Be = sentiment_data['Bearish'].mean()
mid_Be = (max_Be + mean_Be)/2

print ('=========')
print (f"Max Bullish : {max_Bu}")
print (f"Min Bullish : {min_Bu}")
print (f"Avg Bullish : {mean_Bu}")
print (f"Mid Bullish : {mid_Bu}")
print ('=========')
print (f"Max Bearish : {max_Be}")
print (f"Min Bearish : {min_Be}")
print (f"Avg Bearish : {mean_Be}")
print (f"Mid Bearish : {mid_Be}")
print ('=========')

Max Bullish : 0.597531
Min Bullish : 0.203085
Avg Bullish : 0.34880527884615387
Mid Bullish : 0.47316813942307695
Max Bearish : 0.502976
Min Bearish : 0.155556
Avg Bearish : 0.30672322115384615
Mid Bearish : 0.40484961057692304


In [10]:
i=0
consolidated_dict = {}
consolidated_dict = {'Date':['T+1','T+2','T+3','T+4','Bu/Be/Ne', 'Strength']}
max_Bu = sentiment_data['Bullish'].max()


for i in range(len(sentiment_data)-2):
    if sentiment_data['Bullish'][i] == sentiment_data['Bearish'][i] or max(sentiment_data['Bullish'][i], sentiment_data['Bearish'][i], sentiment_data['Neutral'][i]) == sentiment_data['Neutral'][i]:
        consolidated_dict[str(sentiment_data['Date'][i])] = [aux_df['%Change'][1], aux_df['%Change'][2], aux_df['%Change'][3], aux_df['%Change'][4],'Ne',np.nan]
    elif max(sentiment_data['Bullish'][i], sentiment_data['Bearish'][i], sentiment_data['Neutral'][i]) == sentiment_data['Bullish'][i]:
        #consolidated_dict[str(sentiment_data['Date'][i])] = [aux_df['%Change'][1], aux_df['%Change'][2], aux_df['%Change'][3], aux_df['%Change'][4],'Bu']
        if sentiment_data['Bullish'][i] >= mid_Bu:
            consolidated_dict[str(sentiment_data['Date'][i])] = [aux_df['%Change'][1], aux_df['%Change'][2], aux_df['%Change'][3], aux_df['%Change'][4],'Bu','Strong']
        elif sentiment_data['Bullish'][i] >= mean_Bu and sentiment_data['Bullish'][i] < mid_Bu:
            consolidated_dict[str(sentiment_data['Date'][i])] = [aux_df['%Change'][1], aux_df['%Change'][2], aux_df['%Change'][3], aux_df['%Change'][4],'Bu','Moderate']
        else:
            consolidated_dict[str(sentiment_data['Date'][i])] = [aux_df['%Change'][1], aux_df['%Change'][2], aux_df['%Change'][3], aux_df['%Change'][4],'Bu','Weak']        
    elif max(sentiment_data['Bullish'][i], sentiment_data['Bearish'][i], sentiment_data['Neutral'][i]) == sentiment_data['Bearish'][i]:
        #consolidated_dict[str(sentiment_data['Date'][i])] = [aux_df['%Change'][1], aux_df['%Change'][2], aux_df['%Change'][3], aux_df['%Change'][4],'Be']
        if sentiment_data['Bearish'][i] >= mid_Be:
            consolidated_dict[str(sentiment_data['Date'][i])] = [aux_df['%Change'][1], aux_df['%Change'][2], aux_df['%Change'][3], aux_df['%Change'][4],'Be','Strong']
        elif sentiment_data['Bearish'][i] >= mean_Be and sentiment_data['Bearish'][i] < mid_Be:
            consolidated_dict[str(sentiment_data['Date'][i])] = [aux_df['%Change'][1], aux_df['%Change'][2], aux_df['%Change'][3], aux_df['%Change'][4],'Be','Moderate']
        else:
            consolidated_dict[str(sentiment_data['Date'][i])] = [aux_df['%Change'][1], aux_df['%Change'][2], aux_df['%Change'][3], aux_df['%Change'][4],'Be','Weak']        
    i+=1

aux_df = pd.DataFrame.from_dict(consolidated_dict, orient='index')

aux_df.reset_index(inplace=True)
aux_df.columns = ['Date', 'T+1','T+2','T+3','T+4', 'Bu_Be_Ne', 'Strength']
aux_df.drop([0], inplace=True)
aux_df['Date'] = pd.DatetimeIndex(aux_df['Date'])
aux_df.tail()

Date         T+1         T+2        T+3         T+4 Bu_Be_Ne  \
98  2019-11-14  0.00446212  0.00540802  0.0106298  0.00760336       Bu   
99  2019-11-21  0.00446212  0.00540802  0.0106298  0.00760336       Ne   
100 2019-11-29  0.00446212  0.00540802  0.0106298  0.00760336       Ne   
101 2019-12-05  0.00446212  0.00540802  0.0106298  0.00760336       Ne   
102 2019-12-12  0.00446212  0.00540802  0.0106298  0.00760336       Bu   

     Strength  
98   Moderate  
99        NaN  
100       NaN  
101       NaN  
102  Moderate

In [11]:
# Concatenate sentiment_data & aux_df DataFrames into a single DataFrame
sentiment_data.set_index(sentiment_data['Date'], inplace = True)
aux_df.set_index(aux_df['Date'], inplace=True)
joined_data_cols = pd.concat([sentiment_data, aux_df], axis='columns', join='inner')
joined_data_cols.drop(columns = 'Date',inplace = True)
joined_data_cols.head()

Bullish   Neutral   Bearish         T+1         T+2        T+3  \
Date                                                                          
2018-01-04  0.597531  0.246914  0.155556  0.00446212  0.00540802  0.0106298   
2018-01-11  0.486726  0.262537  0.250737  0.00446212  0.00540802  0.0106298   
2018-01-18  0.541133  0.244973  0.213894  0.00446212  0.00540802  0.0106298   
2018-01-25  0.454545  0.305195  0.240260  0.00446212  0.00540802  0.0106298   
2018-02-01  0.447712  0.264706  0.287582  0.00446212  0.00540802  0.0106298   

                   T+4 Bu_Be_Ne  Strength  
Date                                       
2018-01-04  0.00760336       Bu    Strong  
2018-01-11  0.00760336       Bu    Strong  
2018-01-18  0.00760336       Bu    Strong  
2018-01-25  0.00760336       Bu  Moderate  
2018-02-01  0.00760336       Bu  Moderate

In [12]:
joined_data_cols['Bullish'] = joined_data_cols['Bullish'].astype(float)
joined_data_cols['Neutral'] = joined_data_cols['Neutral'].astype(float)
joined_data_cols['Bearish'] = joined_data_cols['Bearish'].astype(float)
joined_data_cols['T+1'] = joined_data_cols['T+1'].astype(float)
joined_data_cols['T+2'] = joined_data_cols['T+2'].astype(float)
joined_data_cols['T+3'] = joined_data_cols['T+3'].astype(float)
joined_data_cols['T+4'] = joined_data_cols['T+4'].astype(float)
joined_data_cols.dtypes

Bullish     float64
Neutral     float64
Bearish     float64
T+1         float64
T+2         float64
T+3         float64
T+4         float64
Bu_Be_Ne     object
Strength     object
dtype: object

In [13]:
joined_data_cols['Bu<->Be'] = joined_data_cols[['Bullish', 'Bearish']].max(axis=1)-joined_data_cols[['Bullish', 'Bearish']].min(axis=1)

In [14]:
joined_data_cols['Bu<->Be'] = joined_data_cols['Bu<->Be'].astype(float)
joined_data_cols

Bullish   Neutral   Bearish       T+1       T+2      T+3  \
Date                                                                    
2018-01-04  0.597531  0.246914  0.155556  0.004462  0.005408  0.01063   
2018-01-11  0.486726  0.262537  0.250737  0.004462  0.005408  0.01063   
2018-01-18  0.541133  0.244973  0.213894  0.004462  0.005408  0.01063   
2018-01-25  0.454545  0.305195  0.240260  0.004462  0.005408  0.01063   
2018-02-01  0.447712  0.264706  0.287582  0.004462  0.005408  0.01063   
...              ...       ...       ...       ...       ...      ...   
2019-11-14  0.407229  0.344578  0.248193  0.004462  0.005408  0.01063   
2019-11-21  0.342432  0.367246  0.290323  0.004462  0.005408  0.01063   
2019-11-29  0.336391  0.360856  0.302752  0.004462  0.005408  0.01063   
2019-12-05  0.317152  0.391586  0.291262  0.004462  0.005408  0.01063   
2019-12-12  0.376316  0.363158  0.260526  0.004462  0.005408  0.01063   

                 T+4 Bu_Be_Ne  Strength   Bu<->Be  
Date                                               
2018-01-04  0.007603       Bu    Strong  0.441975  
2018-01-11  0.007603       Bu    Strong  0.235989  
2018-01-18  0.007603       Bu    Strong  0.327239  
2018-01-25  0.007603       Bu  Moderate  0.214285  
2018-02-01  0.007603       Bu  Moderate  0.160130  
...              ...      ...       ...       ...  
2019-11-14  0.007603       Bu  Moderate  0.159036  
2019-11-21  0.007603       Ne       NaN  0.052109  
2019-11-29  0.007603       Ne       NaN  0.033639  
2019-12-05  0.007603       Ne       NaN  0.025890  
2019-12-12  0.007603       Bu  Moderate  0.115790  

[102 rows x 10 columns]

In [15]:
joined_data_cols.isnull().sum()

Bullish      0
Neutral      0
Bearish      0
T+1          0
T+2          0
T+3          0
T+4          0
Bu_Be_Ne     0
Strength    39
Bu<->Be      0
dtype: int64

In [16]:
joined_data_cols.dropna(inplace = True)

In [17]:
joined_data_cols.count()

Bullish     63
Neutral     63
Bearish     63
T+1         63
T+2         63
T+3         63
T+4         63
Bu_Be_Ne    63
Strength    63
Bu<->Be     63
dtype: int64

In [18]:
joined_data_cols.head()

Bullish   Neutral   Bearish       T+1       T+2      T+3  \
Date                                                                    
2018-01-04  0.597531  0.246914  0.155556  0.004462  0.005408  0.01063   
2018-01-11  0.486726  0.262537  0.250737  0.004462  0.005408  0.01063   
2018-01-18  0.541133  0.244973  0.213894  0.004462  0.005408  0.01063   
2018-01-25  0.454545  0.305195  0.240260  0.004462  0.005408  0.01063   
2018-02-01  0.447712  0.264706  0.287582  0.004462  0.005408  0.01063   

                 T+4 Bu_Be_Ne  Strength   Bu<->Be  
Date                                               
2018-01-04  0.007603       Bu    Strong  0.441975  
2018-01-11  0.007603       Bu    Strong  0.235989  
2018-01-18  0.007603       Bu    Strong  0.327239  
2018-01-25  0.007603       Bu  Moderate  0.214285  
2018-02-01  0.007603       Bu  Moderate  0.160130

In [19]:
max_Sp = joined_data_cols['Bu<->Be'].max()
min_Sp = joined_data_cols['Bu<->Be'].min()
mean_Sp = joined_data_cols['Bu<->Be'].mean()
mid_Sp = (max_Sp + mean_Sp)/2

print ('=========')
print (f"Max Spread : {max_Sp}")
print (f"Min Spread : {min_Sp}")
print (f"Avg Spread : {mean_Sp}")
print (f"Mid Spread : {mid_Sp}")
print ('=========')

Max Spread : 0.441975
Min Spread : 0.008772000000000002
Avg Spread : 0.14499847619047626
Mid Spread : 0.2934867380952381


In [20]:
i=0
spread_dict = {}
joined_data_cols.reset_index(inplace = True)
for i in range(len(joined_data_cols)):
    if joined_data_cols['Bu_Be_Ne'][i] == 'Bu' and joined_data_cols['T+1'][i] > 0:
        if joined_data_cols['Bu<->Be'][i] >= mid_Sp:
            spread_dict[joined_data_cols['Date'][i]] = ['High', 'Y']
        elif joined_data_cols['Bu<->Be'][i] >= mean_Sp and joined_data_cols['Bu<->Be'][i] < mid_Sp:
            spread_dict[joined_data_cols['Date'][i]] = ['Mid', 'Y']
        else:
            spread_dict[joined_data_cols['Date'][i]] = ['Low','Y']
    elif joined_data_cols['Bu_Be_Ne'][i] == 'Bu' and joined_data_cols['T+1'][i] <= 0:
        if joined_data_cols['Bu<->Be'][i] >= mid_Sp:
            spread_dict[joined_data_cols['Date'][i]] = ['High', 'N']
        elif joined_data_cols['Bu<->Be'][i] >= mean_Sp and joined_data_cols['Bu<->Be'][i] < mid_Sp:
            spread_dict[joined_data_cols['Date'][i]] = ['Mid', 'N']
        else:
            spread_dict[joined_data_cols['Date'][i]] = ['Low','N']
    elif joined_data_cols['Bu_Be_Ne'][i] == 'Be' and joined_data_cols['T+1'][i] < 0:
        if joined_data_cols['Bu<->Be'][i] >= mid_Sp:
            spread_dict[joined_data_cols['Date'][i]] = ['High', 'Y']
        elif joined_data_cols['Bu<->Be'][i] >= mean_Sp and joined_data_cols['Bu<->Be'][i] < mid_Sp:
            spread_dict[joined_data_cols['Date'][i]] = ['Mid', 'Y']
        else:
            spread_dict[joined_data_cols['Date'][i]] = ['Low','Y']
    elif joined_data_cols['Bu_Be_Ne'][i] == 'Be' and joined_data_cols['T+1'][i] >= 0:
        if joined_data_cols['Bu<->Be'][i] >= mid_Sp:
            spread_dict[joined_data_cols['Date'][i]] = ['High', 'N']
        elif joined_data_cols['Bu<->Be'][i] >= mean_Sp and joined_data_cols['Bu<->Be'][i] < mid_Sp:
            spread_dict[joined_data_cols['Date'][i]] = ['Mid', 'N']
        else:
            spread_dict[joined_data_cols['Date'][i]] = ['Low','N']
    
    if joined_data_cols['Bu_Be_Ne'][i] == 'Bu' and joined_data_cols['T+2'][i] > 0:
        spread_dict[joined_data_cols['Date'][i]].append('Y')
    elif joined_data_cols['Bu_Be_Ne'][i] == 'Bu' and joined_data_cols['T+2'][i] <= 0:
        spread_dict[joined_data_cols['Date'][i]].append('N')
    elif joined_data_cols['Bu_Be_Ne'][i] == 'Be' and joined_data_cols['T+2'][i] < 0:
        spread_dict[joined_data_cols['Date'][i]].append('Y')
    elif joined_data_cols['Bu_Be_Ne'][i] == 'Be' and joined_data_cols['T+2'][i] >= 0:
        spread_dict[joined_data_cols['Date'][i]].append('N')
    
    if joined_data_cols['Bu_Be_Ne'][i] == 'Bu' and joined_data_cols['T+3'][i] > 0:
        spread_dict[joined_data_cols['Date'][i]].append('Y')
    elif joined_data_cols['Bu_Be_Ne'][i] == 'Bu' and joined_data_cols['T+3'][i] <= 0:
        spread_dict[joined_data_cols['Date'][i]].append('N')
    elif joined_data_cols['Bu_Be_Ne'][i] == 'Be' and joined_data_cols['T+3'][i] < 0:
        spread_dict[joined_data_cols['Date'][i]].append('Y')
    elif joined_data_cols['Bu_Be_Ne'][i] == 'Be' and joined_data_cols['T+3'][i] >= 0:
        spread_dict[joined_data_cols['Date'][i]].append('N')
    
    if joined_data_cols['Bu_Be_Ne'][i] == 'Bu' and joined_data_cols['T+4'][i] > 0:
        spread_dict[joined_data_cols['Date'][i]].append('Y')
    elif joined_data_cols['Bu_Be_Ne'][i] == 'Bu' and joined_data_cols['T+4'][i] <= 0:
        spread_dict[joined_data_cols['Date'][i]].append('N')
    elif joined_data_cols['Bu_Be_Ne'][i] == 'Be' and joined_data_cols['T+4'][i] < 0:
        spread_dict[joined_data_cols['Date'][i]].append('Y')
    elif joined_data_cols['Bu_Be_Ne'][i] == 'Be' and joined_data_cols['T+4'][i] >= 0:
        spread_dict[joined_data_cols['Date'][i]].append('N')
        
    i+=1
    
aux_df = pd.DataFrame.from_dict(spread_dict, orient='index')

aux_df.reset_index(inplace=True)
aux_df.columns = ['Date', 'Spread', 'MatchT+1', 'MatchT+2', 'MatchT+3', 'MatchT+4']
aux_df['Date'] = pd.DatetimeIndex(aux_df['Date'])

aux_df.set_index(aux_df['Date'], inplace=True)
aux_df.drop(columns = 'Date', inplace = True)
aux_df.tail()

Spread MatchT+1 MatchT+2 MatchT+3 MatchT+4
Date                                                 
2019-10-03    Mid        N        N        N        N
2019-10-10    Mid        N        N        N        N
2019-11-07    Mid        Y        Y        Y        Y
2019-11-14    Mid        Y        Y        Y        Y
2019-12-12    Low        Y        Y        Y        Y

In [21]:
#Concatenate joined_data_cols & aux_df
joined_data_cols.set_index(joined_data_cols['Date'], inplace=True)
joined_data_cols = pd.concat([joined_data_cols, aux_df], axis='columns', join='inner')
joined_data_cols.drop(columns = 'Date',inplace = True)
joined_data_cols.tail()

Bullish   Neutral   Bearish       T+1       T+2      T+3  \
Date                                                                    
2019-10-03  0.213740  0.391858  0.394402  0.004462  0.005408  0.01063   
2019-10-10  0.203085  0.357326  0.439589  0.004462  0.005408  0.01063   
2019-11-07  0.403023  0.357683  0.239295  0.004462  0.005408  0.01063   
2019-11-14  0.407229  0.344578  0.248193  0.004462  0.005408  0.01063   
2019-12-12  0.376316  0.363158  0.260526  0.004462  0.005408  0.01063   

                 T+4 Bu_Be_Ne  Strength   Bu<->Be Spread MatchT+1 MatchT+2  \
Date                                                                         
2019-10-03  0.007603       Be  Moderate  0.180662    Mid        N        N   
2019-10-10  0.007603       Be    Strong  0.236504    Mid        N        N   
2019-11-07  0.007603       Bu  Moderate  0.163728    Mid        Y        Y   
2019-11-14  0.007603       Bu  Moderate  0.159036    Mid        Y        Y   
2019-12-12  0.007603       Bu  Moderate  0.115790    Low        Y        Y   

           MatchT+3 MatchT+4  
Date                          
2019-10-03        N        N  
2019-10-10        N        N  
2019-11-07        Y        Y  
2019-11-14        Y        Y  
2019-12-12        Y        Y

In [22]:
joined_data_cols.groupby(['Bu_Be_Ne','Strength','Spread','MatchT+4']).count()

Bullish  Neutral  Bearish  T+1  T+2  T+3  \
Bu_Be_Ne Strength Spread MatchT+4                                             
Be       Moderate Low    N              11       11       11   11   11   11   
                  Mid    N               2        2        2    2    2    2   
         Strong   Low    N               3        3        3    3    3    3   
                  Mid    N              10       10       10   10   10   10   
Bu       Moderate Low    Y              20       20       20   20   20   20   
                  Mid    Y              13       13       13   13   13   13   
         Strong   High   Y               2        2        2    2    2    2   
                  Mid    Y               2        2        2    2    2    2   

                                   T+4  Bu<->Be  MatchT+1  MatchT+2  MatchT+3  
Bu_Be_Ne Strength Spread MatchT+4                                              
Be       Moderate Low    N          11       11        11        11        11  
                  Mid    N           2        2         2         2         2  
         Strong   Low    N           3        3         3         3         3  
                  Mid    N          10       10        10        10        10  
Bu       Moderate Low    Y          20       20        20        20        20  
                  Mid    Y          13       13        13        13        13  
         Strong   High   Y           2        2         2         2         2  
                  Mid    Y           2        2         2         2         2

In [23]:
joined_data_cols.groupby('Bu_Be_Ne').count()

Bullish  Neutral  Bearish  T+1  T+2  T+3  T+4  Strength  Bu<->Be  \
Bu_Be_Ne                                                                     
Be             26       26       26   26   26   26   26        26       26   
Bu             37       37       37   37   37   37   37        37       37   

          Spread  MatchT+1  MatchT+2  MatchT+3  MatchT+4  
Bu_Be_Ne                                                  
Be            26        26        26        26        26  
Bu            37        37        37        37        37